<a href="https://colab.research.google.com/github/jakobzhao/data-classification/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geospatial Data Classification

This lab helps you critically understand how **geospatial data is classified and visualized**, building directly on Wilson (2011) and Kwan (2012). You will:

- Retrieve real-world facility data from OpenStreetMap (OSM)
- Apply **two different classification schemes**
- Visualize both schemes **side-by-side** on interactive maps
- Reflect on how categorization shapes what we see—and what we don't
- Learn how to make a **counter-map**.

> **Guiding question**: How do classification schemes shape urban narratives, planning decisions, and power relations?

In [1]:
# Install required packages
!pip install osmnx folium geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 3.6 MB/s eta 0:00:00


In [2]:
# Import libraries
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import DualMap

We retrieve public facility data using the following **OSM tags**:

- `amenity`: includes services such as `school`, `library`, `parking`, `fuel`  
- `building`: includes physical infrastructure like `school`, `library`, `public`  
- `leisure`: includes recreational places like `park`  

🔗 Learn more about OSM tags here:  https://wiki.openstreetmap.org/wiki/Map_features

In [3]:
# Download facility data from OSM for Seattle
place_name = "Seattle, Washington, USA"
tags = {
    "amenity": ["school", "library", "fuel", "parking"],
    "leisure": ["park"],
    "building": ["public"]
}
pois = ox.features_from_place(place_name, tags)
pois = pois[pois.geometry.type == "Point"]  # Only use point features

In this lab, we will classify the public facilities into three categories. For example, the following categorization reflects a **dominant urban governance perspective**:

| Category| Examples (OSM tags)| Implied Logic / Social Meaning |
|---------------------|-----------------------------|--------------------|
| `community_asset`   | `school`, `library`| Viewed as valuable public infrastructure and resources worth preserving |
| `neutral`           | `fuel`, `parking`           | Functionally useful, but politically and socially "depoliticized" |
| `deficit`           | `Other` / `undefined` / `ambiguous uses` | Often seen as underutilized, marginal, or needing intervention |

- **"Assets"** are those that receive investment, maintenance, and visibility  
- **"Deficits"** are often seen as problems to be managed, removed, or re-zoned  
- **"Neutral"** places are frequently assumed to be value-free, yet they shape daily life and reflect planning priorities

Below, we will categorize the places under two different perspectives.


In [4]:
# Classify facilities under two different perspectives
def classify_planner(row):
    if row.get("amenity") in ["school", "library", "park"]:
        return "community_asset"
    elif row.get("amenity") in ["fuel", "parking"]:
        return "neutral"
    else:
        return "deficit"

def classify_community(row):
    if row.get("amenity") in ["school", "library", "park"]:
        return "community_asset"
    elif row.get("amenity") in ["fuel"]:
        return "deficit"
    else:
        return "neutral"

pois["category_planner"] = pois.apply(classify_planner, axis=1)
pois["category_community"] = pois.apply(classify_community, axis=1)

## Example for Discussion

- A school is clearly an `asset` from a public investment perspective. But is a community garden also an asset?
- A gas station is `neutral` in this scheme—but what about its environmental or public health impact?
- A parking lot supports accessibility—but also displaces green space. Should it be `deficit` instead?
- An abandoned building may be labeled `deficit`, yet might hold cultural, historic, or creative potential.


In [5]:
# Create side-by-side map comparison
color_map = {
    "community_asset": "green",
    "neutral": "blue",
    "deficit": "red"
}

dual_map = DualMap(location=[47.61, -122.33], zoom_start=12)

# Add to left map (planner view)
for _, row in pois.iterrows():
    folium.CircleMarker(
        location=(row.geometry.y, row.geometry.x),
        radius=5,
        color=color_map.get(row["category_planner"], "gray"),
        fill=True,
        fill_color=color_map.get(row["category_planner"], "gray"),
        popup=f"{row.get('amenity', 'unknown')} ({row['category_planner']})"
    ).add_to(dual_map.m1)

# Add to right map (community view)
for _, row in pois.iterrows():
    folium.CircleMarker(
        location=(row.geometry.y, row.geometry.x),
        radius=5,
        color=color_map.get(row["category_community"], "gray"),
        fill=True,
        fill_color=color_map.get(row["category_community"], "gray"),
        popup=f"{row.get('amenity', 'unknown')} ({row['category_community']})"
    ).add_to(dual_map.m2)

dual_map

## Final Reflection

You have now visualized the same geospatial data under two different classification perspectives:

- **Planner View**: prioritizes formal civic infrastructure.
- **Community View**: includes alternative or under-valued sites.


In this final task, you will **design a counter-map** that centers a marginalized group’s experience (e.g., unhoused people, immigrants, youth, disabled residents).  

Ask: What counts as a “resource” from their perspective? What’s missing from official views?

### Part 1: Define Your Own Classification Logic

- Propose **your own categories** (e.g., `safe_space`, `environmental_risk`, `youth_value`, `invisible_infrastructure`, etc.)
- You may define **2–4 categories** and choose how to group or differentiate them.
- Justify your choices:  
  > What values, communities, or problems are you trying to highlight through your classification?


### Part 2: Implement and Visualize

- Write a custom classification function in Python.
- Apply it to the OSM facility data.
- Create a **single interactive map** with Folium (or DualMap if comparing two logics).
- Use color coding, popups, or symbols to make your perspective visible.

---

### Part 3: Reflection and Communication

Answer the following questions (300–500 words):

1. **What logic guided your classification?**  
   How is it different from the planner or community view provided in this lab?

2. **What changes appeared on your map?**  
   Which spaces became more visible or important under your scheme?

3. **How would you present your map to a specific audience** (e.g., city planners, youth groups, housing advocates)?  
   Would you change anything about how it's designed or explained?

Your categories don’t need to be perfect or comprehensive—they should provoke insight and reflection. Please Consider how **color, interactivity, and narrative** shape the interpretation of your map. As we did in the previous lab, please creatively use AI to help you finish this assignment.